In [1]:
!nvidia-smi

Thu May 18 15:15:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Packages

In [2]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.8 MB/s eta 0:00:00


In [4]:
!pip install hanja
import os
import torch
import numpy as np
import pandas as pd
import re
import hanja

from transformers import Trainer, TrainingArguments
from transformers import ElectraForSequenceClassification, ElectraTokenizer
from transformers import BertForSequenceClassification, BertTokenizerFast

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import seaborn as sn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.0/265.0 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.3/213.3 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.2/228.2 kB 24.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of coverage[toml] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.2/228.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 2

## Load News Data & Process

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/BalancedNewsCorpus_train.csv')
test = pd.read_csv('/content/drive/MyDrive/BalancedNewsCorpus_test.csv')

In [ ]:
print('train data: {}, test data: {}'.format(len(train),len(test)))

train data: 9000, test data: 1800


In [ ]:
train.head(3)

filename      date NewsPaper Topic  \
0  NLRW1900000141  20170324      부산일보   스포츠   
1  NPRW1900000003  20110209   한국경제신문사    정치   
2  NLRW1900000144  20100406      영남일보    사회   

                                                News  
0  <p> 야구 종가, 마침내 정상에 서다 </p> <p> '야구 종가' 미국이 푸에르...  
1  <p> 외통위 27명중 15명 "FTA 추가협상안만 처리" </p> <p> 국회 외...  
2  <p> 한나라 "地選후보, 희망연대 당원 구함" 공천변수 작용 주목 </p> <p>...

### Preprocessing

In [ ]:
def cleanse_sentence(sent): 
    
    # 문장 내 한자 제거
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sent) is not None:
        sent = hanja.translate(sent, 'substitution')
        
    # 문장 내 특수기호 제거
    sent = sent.replace('<p>', '\n').replace('</p>', '\n')
    sent = (r'\[[^\]]+\]|\([^\)]+\)|\<[^\>]+\>', '', str(sent))
    sent = re.sub("[.,!?||]", " ", str(sent))
    sent = re.sub(r'[^가-힣|\s]', '', str(sent))
    sent = re.sub(r'\s+', ' ', str(sent)) 
    
    return sent.rstrip()

In [ ]:
train['News'] = train['News'].map(lambda x: cleanse_sentence(x))
test['News'] = test['News'].map(lambda x: cleanse_sentence(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Possible set union at position 5
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
train.head(3)

filename      date NewsPaper Topic  \
0  NLRW1900000141  20170324      부산일보   스포츠   
1  NPRW1900000003  20110209   한국경제신문사    정치   
2  NLRW1900000144  20100406      영남일보    사회   

                                                News  
0   야구 종가 마침내 정상에 서다 야구 종가 미국이 푸에르토리코를 누르고 월드베이스볼...  
1   외통위 명중 명 추가협상안만 처리 국회 외교통상통일위원회 소속 의원 명 중 명이 ...  
2   한나라 지선후보 희망연대 당원 구함 공천변수 작용 주목 오늘까지 추가 모집 오는 ...

### Labeling

In [ ]:
# Check values
train['Topic'].value_counts()

스포츠      1000
정치       1000
사회       1000
문화       1000
미용/건강    1000
IT/과학    1000
생활       1000
경제       1000
연예       1000
Name: Topic, dtype: int64

In [ ]:
test['Topic'].value_counts()

IT/과학    200
경제       200
문화       200
미용/건강    200
사회       200
생활       200
스포츠      200
연예       200
정치       200
Name: Topic, dtype: int64

In [ ]:
topics = train['Topic'].value_counts().keys().tolist()

In [ ]:
label_map = {v:i for i, v in enumerate(topics)}

In [ ]:
label_map

{'IT/과학': 5,
 '경제': 7,
 '문화': 3,
 '미용/건강': 4,
 '사회': 2,
 '생활': 6,
 '스포츠': 0,
 '연예': 8,
 '정치': 1}

In [ ]:
train['Label'] = train['Topic'].map(lambda x: label_map[x])
test['Label'] = test['Topic'].map(lambda x: label_map[x])

In [ ]:
train.head(3)

filename      date NewsPaper Topic  \
0  NLRW1900000141  20170324      부산일보   스포츠   
1  NPRW1900000003  20110209   한국경제신문사    정치   
2  NLRW1900000144  20100406      영남일보    사회   

                                                News  Label  
0   야구 종가 마침내 정상에 서다 야구 종가 미국이 푸에르토리코를 누르고 월드베이스볼...      0  
1   외통위 명중 명 추가협상안만 처리 국회 외교통상통일위원회 소속 의원 명 중 명이 ...      1  
2   한나라 지선후보 희망연대 당원 구함 공천변수 작용 주목 오늘까지 추가 모집 오는 ...      2

## Load Model

In [ ]:

# model = BertForSequenceClassification.from_pretrained('snunlp/KR-Medium', num_labels=9)
# tokenizer = BertTokenizerFast.from_pretrained('snunlp/KR-Medium')


model = ElectraForSequenceClassification.from_pretrained('snunlp/KR-ELECTRA-discriminator', num_labels=9)
tokenizer = ElectraTokenizer.from_pretrained('snunlp/KR-ELECTRA-discriminator')


Some weights of the model checkpoint at snunlp/KR-ELECTRA-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-ELECTRA-discriminator and are newly initialized: ['classifier.dense

### Check max length

In [ ]:
lens = []
for sent in train['News']:
  lens.append(len(tokenizer.tokenize(sent)))

In [ ]:
print('avg : {}, max : {}'.format(sum(lens)/len(lens), max(lens)))

avg : 535.0174444444444, max : 722


## Build Dataset

In [ ]:
#Make Dataset Class
class news_data(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer):
        super().__init__()
        self.label = torch.tensor(dataset['Label']).long()
        features = tokenizer([str(x) for x in dataset.News], padding=True, truncation=True,max_length=512)
        self.input = torch.tensor(features['input_ids'])
        self.mask = torch.tensor(features['attention_mask'])
    
    def __len__(self):
        return len(self.input)

    def __getitem__(self, index):
        return {'input_ids': self.input[index], 'attention_mask': self.mask[index], 'label':self.label[index]}

In [ ]:
train_data = news_data(train, tokenizer)
test_data = news_data(test, tokenizer)

In [ ]:
print('train data: {}, test data: {}'.format(len(train_data),len(test_data)))

train data: 9000, test data: 1800


## Parameter setting

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
#Trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3, 
    per_device_train_batch_size=8, # decrease batch size to avoid OOM error
    per_device_eval_batch_size=8,
    warmup_steps=300,
    weight_decay=0.01,
    save_steps=1000,
    logging_dir='./logs'
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)

## Train

In [ ]:
#Train Model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3375


Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3000/config.json
Model weights saved in ./results/checkpoint-3000/pytorch_model.bin


## Evaluate

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1800
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.8866666666666667,
 'eval_f1': 0.8859523141335739,
 'eval_loss': 0.5173348784446716,
 'eval_precision': 0.8862258365962591,
 'eval_recall': 0.8866666666666667,
 'eval_runtime': 58.3755,
 'eval_samples_per_second': 30.835,
 'eval_steps_per_second': 3.854}

In [ ]:
# 나중을 위해 모델 저장
# trainer.save_model('kr-electra-news')